This notebook is from March 2017 (*before* the PSF workshop at STScI) 

In [ ]:
... imports ...

from photutils.psf import build_psf

In [ ]:
# do whatever data loading is required. 
...
list_of_ndd_obj = load_data(...)  
# The end result should be a list of NDData objects containing all the 
# dithers in one filter for that observation

# Baseline object

To follow the rest of the PSF machinery, the structure here is to create a class where all the parameters get set, and then run that class on the data.

## Creating the builder 

In [ ]:
psf_builder = PSFBuilder(model=Gaussian2D())  
# while most parameters should have defaults, it may be that the `model` is special and should have to be specified explicitly

You use the keywords to specify specific algorithms for sub-stages of the process. E.g., if a specific fitter is desired:

In [ ]:
psf_builder = PSFBuilder(model=EPSF(), fitter=EPPSFFitter(...))

Some parameters might be intrinsic to the building process, which can just be lone keywords:

In [ ]:
psf_builder = PSFBuilder(model=Gaussian2D(), max_psf_iters=5)

Whereas parameters specific to a given step can be given as arguments to those objects:

In [ ]:
# this would mean that in the 
psf_builder = PSFBuilder(model=Gaussian2D(), recenterer=FirstMoment(sigclip=True))

For customizable stages, photutils should provide specific cases (like `FirstMoment` or `OFilter`), but these should be callables, so that the user can do the simple thing of just something like:

In [ ]:
def custom_recenterer(star_image, x0, y0):
    # you've decided for some reason the first moment gives something that's always .1 pixels off...
    x0, y0 = FirstMoment()(star_image, x0, y0)
    return x0 + .1, y0 + .1
    
psf_builder = PSFBuilder(model=Gaussian2D(), recenterer=custom_recenterer) 

## Running the builder 

In the below, `psf_stars` is an astropy table that looks like what the star-finder outputs.  The key bits the builder will look for are either:
* a pair of columns called `xcentroid`/`ycentroid` (which will be interpreted as x/y in the *first* image of the list - so then `wcs` is only required if there's more than one image), or 
* `world_x`/`world_y`, which will then use the `wcs` for all of the images to convert to pixel space.

In [ ]:
psf_model, psf_stars_perdither = psf_builder(list_of_ndd_obj, psf_stars)

The key end result is the `psf_model`, which is a 2D Astropy model that maps x,y to the PSF flux.  This is the same  current `photutils.psf` models like `FittableImageModel`. Should be normalized so that if the `flux` parameter is 1, the PSF's total flux is 1. Note that this *might* not mean the ePSF sums to 1: this allows aperture corrections to be folded into the model.

The `psf_stars_perdither` is a list of `Table`s, one for each `list_of_ndd_obj` NDData's. It's the same shape as psf_stars, but would carry with it information like the exact pixel coordinates for *that* dither (computed from the wcs), how much that star in that dither was weighted in the final PSF, etc.

# Putting it together

This demonstrates how the machinery would fit together with the whole PSF-photometry process (without any quality-checking in the loop for simplicity)

In [ ]:
fwhm = 3.0  # a guess that you'd need to input based on seeing, or knowledge that it's JWST data

combined = reproject.drizzle(list_of_ndd_obj)  
# drizzle functionality doesn't exist yet in `reproject`, but it is planned...

mean, median, std = sigma_clipped_stats(combined, sigma=3.0, iters=5)

psf_stars =  IRAFStarFinder(fwhm=fwhm, threshold=50*std)(combined)

In [ ]:
psf_builder = PSFBuilder(model=EPSF() fitter=EPSFFitter()) 
psf_model, psf_stars_perdither =  psf_builder(list_of_ndd_obj, psf_stars)

In [ ]:
photometry = DAOPhotPSFPhotometry(crit_separation=10, threshold=5*std, fwhm=fwhm, 
                                  psf_model=psf_model, 
                                  fitshape=(11, 11))
# currently, DAOPhotPSFPhotometry only supports running on a single image
result_tab = photometry(combined)

Now `result_tab` contains the photometry information desired. Specifically, it contains `x_fit`, `y_fit`, and `flux_fit` columns, as well as `x_0`, `y_0`, and `flux_0` (the initial guesses).

Later on, when we have the per-dither fitting working, it would be:

In [ ]:
result_tab = photometry(list_of_ndd_obj)

And `result_tab` would *also* have  columns that provide information on specific dithers.

### Quality checks 

A standing question is how to populate the table with quality checks.  Should that be considered part of the photometry process?  If so, the `result_tab` would already have them.  Alternatively, it could be a separate step, a la:

In [ ]:
#DAOPhotStatistics does not exist at the time of this writing, so this is just a concept
result_tab = DAOPhotStatistics(sigma_clip=5)(result_tab, combined, psf_model)  

and the new `result_tab` would gain `chi` and `sharp` statistics

# Heuristic high-level function (extreme stretch goal)

A `build_psf` function here would have to make a lot of assumptions about the data, etc.  So the function would probably need to contain a lot of heuristic bits to guess detection thresholds,  etc.  But the idea would be that it would pull together all of the above into a one-stop-shop thing.

In [ ]:
psf_model = build_psf(list_of_ndd_obj)